# A Comprehensive Tutorial on Rasterio

Rasterio is a Python library for working with geospatial raster data, built atop GDAL, but with a Pythonic API that integrates seamlessly with NumPy. This tutorial introduces key raster operations using Rasterio, assumes familiarity with Python, NumPy, GDAL, and Matplotlib, and includes comparisons with GDAL and a bonus section showcasing some cool Rasterio capabilities.

## Table of Contents

- [Getting Started with Rasterio](#1-getting-started-with-rasterio)
- [Reading Raster Data](#2-reading-raster-data)
- [Writing and Updating Rasters](#3-writing-and-updating-rasters)
- [Masking and Cropping](#4-masking-and-cropping)
- [Reprojecting Rasters](#5-reprojecting-rasters)
- [Performing Zonal Statistics](#6-performing-zonal-statistics)
- [Comparing GDAL with Rasterio](#7-comparing-gdal-with-rasterio) 
- [Bonus: Cool Tricks with Rasterio](#8-bonus-cool-tricks-with-rasterio)

## 1. Getting Started with Rasterio

### Installation

In [ ]:
!pip install rasterio # for windows !(Get-Command python).Path -m pip install rasterio

### Importing Rasterio

In [ ]:
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt

## 2. Reading Raster Data

### Opening a Raster File

In [ ]:
with rasterio.open('example.tif') as src:
    print(src.profile)  # Metadata
    data = src.read(1)  # Read the first band

### Visualizing Raster Data

In [ ]:
plt.imshow(data, cmap='viridis')
plt.colorbar(label='Pixel Value')
plt.title('Raster Band Visualization')
plt.show()

### Reading Raster as an Array

In [ ]:
array = src.read(1)
print(array.shape)

## 3. Writing and Updating Rasters

### Creating a New Raster

In [ ]:
with rasterio.open(
    'new_raster.tif',
    'w',
    driver='GTiff',
    height=data.shape[0],
    width=data.shape[1],
    count=1,
    dtype=data.dtype,
    crs=src.crs,
    transform=src.transform
) as dst:
    dst.write(data, 1)

### Updating an Existing Raster

In [ ]:
with rasterio.open('existing_raster.tif', 'r+') as dst:
    dst.write(array * 2, 1)  # Example modification

## 4. Masking and Cropping

### Applying a Mask

In [ ]:
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd

# Define a bounding box
bbox = box(minx, miny, maxx, maxy)
geo = gpd.GeoDataFrame({'geometry': [bbox]}, crs=src.crs)

# Apply mask
with rasterio.open('example.tif') as src:
    out_image, out_transform = mask(src, geo.geometry, crop=True)
    show(out_image[0])


### Cropping to a Region

In [ ]:
with rasterio.open('example.tif') as src:
    window = src.window(minx, miny, maxx, maxy)
    cropped = src.read(1, window=window)
    plt.imshow(cropped)

## 5. Reprojecting Rasters

### Reproject to Another CRS

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'EPSG:4326'  # Example target CRS
with rasterio.open('example.tif') as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({'crs': dst_crs, 'transform': transform, 'width': width, 'height': height})

    with rasterio.open('reprojected.tif', 'w', **kwargs) as dst:
        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest)

## 6. Performing Zonal Statistics

### Calculate Mean Value per Zone

In [ ]:
from rasterstats import zonal_stats
import geopandas as gpd

# Load zones as GeoDataFrame
zones = gpd.read_file('zones.shp')

# Perform zonal statistics
stats = zonal_stats(zones, 'example.tif', stats=['mean', 'max', 'min'])
print(stats)

## 7. Comparing GDAL with Rasterio

<!-- Feature	GDAL	Rasterio
Ease of Use	C-like API, requires more boilerplate	Pythonic API, intuitive and readable
Dependencies	Heavily tied to C libraries	Built on GDAL but adds Pythonic simplicity
Performance	Optimized for speed	Slightly slower due to abstraction
Community	Long history, large user base	Growing, vibrant Python community
Visualization	No direct tools	Built-in plotting functions -->

## 8. Bonus: Cool Tricks with Rasterio

### Calculating NDVI

In [ ]:
with rasterio.open('red_band.tif') as red, rasterio.open('nir_band.tif') as nir:
    red_band = red.read(1).astype('float32')
    nir_band = nir.read(1).astype('float32')
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    
    plt.imshow(ndvi, cmap='RdYlGn')
    plt.colorbar(label='NDVI')
    plt.title('Normalized Difference Vegetation Index')
    plt.show()

### Stacking Multiple Bands

In [ ]:
with rasterio.open('example.tif') as src1, rasterio.open('example2.tif') as src2:
    stack = np.stack([src1.read(1), src2.read(1)])
    with rasterio.open(
        'stacked.tif', 'w',
        driver='GTiff',
        height=stack.shape[1],
        width=stack.shape[2],
        count=stack.shape[0],
        dtype=stack.dtype,
        crs=src1.crs,
        transform=src1.transform
    ) as dst:
        dst.write(stack)

### Rasterizing a Vector

In [ ]:
from rasterio.features import rasterize
from shapely.geometry import mapping

shapes = [mapping(geometry) for geometry in zones.geometry]
raster = rasterize(shapes, out_shape=src.shape, transform=src.transform)
plt.imshow(raster)